Notebook to generate data issues

In [1]:
# 1 missing_columns => one or some of the csv files are missing a required column.
# 2 missing_values => one or more values in a row are nulls.
# 3 unknown_categorical_values => expected value "dinner", "lunch", new value: "drinks".
# 4 unknown_numeric_values => wrong value for a feature, e.g:-1 for tips.
# 5 bad_data_type_values => string value in a numerical column and vice versa.
# 6 bad_csv_encoding => having unreadable character in CSV file e.g: à, ù, Ö, Ü.
# 7 bad_delimiter => one or some csv files are seperated by other delimiter e.g: TSV using \t.
# 8 missing_header => one or more header are missing.

In [1]:
import pandas as pd

data2 = {
    "total_bill": [16.99, 10.34, -21.01, 23.68, 24.59],
    "tips": [1.01, None, 3.5, 3.31, 3.61],
    "sex": ["Female", "Male", "Male", "Mäle", "Female"],
    "smoker": [False, False, True, True, True],
    "day": ["Sat", "Sat", "Sun", "Sun", "Mon"],
    "time": ["Dinner", "Lunch", "Lunch", "Dinner", "Drinks"],
    "size": ["two", 3, 3, 200, 4],
}

df_missing_column = pd.read_csv(
    "../../tests/resources/test_folder_1/db_errors_test_folder/test_missing_column.csv"
)
df_error2 = pd.DataFrame(data2)
df_bad_csv = pd.read_csv(
    "../../tests/resources/test_folder_1/db_errors_test_folder/test_bad_csv_errors.csv"
)
print(df_missing_column)
print()
print(df_error2)
print()
print(df_bad_csv)

    tips   sex  smoker day    time  size
0  16.99  1.01  Female  No  Dinner     2
1  10.34  1.66    Male  No  Dinner     3
2  21.01  3.50    Male  No  Dinner     3
3  23.68  3.31    Male  No   Lunch     2
4  24.59  3.61  Female  No   Lunch     4

   total_bill  tips     sex  smoker  day    time size
0       16.99  1.01  Female   False  Sat  Dinner  two
1       10.34   NaN    Male   False  Sat   Lunch    3
2      -21.01  3.50    Male    True  Sun   Lunch    3
3       23.68  3.31    Mäle    True  Sun  Dinner  200
4       24.59  3.61  Female    True  Mon  Drinks    4

  total_bill\ttip\tsex\tsmoker\t\ttime\tsize
0    16.99\t1.01\tFemæle\tNo\tSun\tDinner\t2
1      10.34\t1.66\tMale\tNo\tSun\tDinner\t3
2       21.01\t3.5\tMàle\tNo\tSun\tDinner\t3
3      23.68\t3.31\tMale\tNo\tSun\tDinner\t2
4    24.59\t3.61\tFemale\tNo\tSun\tDinner\t4


In [4]:
# set up empty error dict
empty_errors = {
    "missing_header": None,
    "bad_delimiter": None,
    "bad_csv_encoding": None,
    "missing_columns": None,
    "missing_values": None,
    "unknown_categorical_values": None,
    "unknown_numeric_values": None,
    "bad_data_type_values": None,
}

In [ ]:
def check_delimiter(filepath):
    with open(filepath, "r", encoding="utf-8") as file:
        header_line = file.readline().strip()
    delimiters = [",", "\t", ";", "|", " "]
    delimiter_counts = {d: header_line.count(d) for d in delimiters}
    detected_delimiter = max(delimiter_counts, key=delimiter_counts.get)
    return detected_delimiter

In [ ]:
def check_missing_headers(filepath, delimiter = ","):
    expected_headers = ["total_bill", "tip", "sex", "smoker", "day", "time", "size"]
    with open(filepath, "r", encoding="utf-8") as file:
        header_line = file.readline().strip()
    detected_headers = [h.strip() for h in header_line.split(delimiter)]
    missing_headers = list(set(expected_headers) - set(detected_headers))
    return missing_headers

In [ ]:
def check_missing_columns(filepath, delimiter = ","):
    with open(filepath, "r", encoding="utf-8") as file:
        header_line = file.readline().strip()
        data_line = file.readline().strip()
    num_columns = len(data_line.split(delimiter))
    if num_columns < 7:
        return "Missing columns, need manual inspect"

In [79]:
def check_bad_encode(filepath, d='\t'):
    df = pd.read_csv(filepath, sep=d)
    bad_encode_values = {}
    for col in df.columns:
        rows = []
        for index, value in df[col].items():
            try:
                value.encode('utf-8')
            except AttributeError:
                value = str(value)
            except UnicodeEncodeError:
                rows.append(index)
        if rows:
            bad_encode_values[col] = rows
    return bad_encode_values

In [80]:
print(check_bad_encode("../../tests/resources/test_folder_1/db_errors_test_folder/test_bad_csv_errors.csv"))
df3 = pd.read_csv("../../tests/resources/test_folder_1/db_errors_test_folder/test_bad_csv_errors.csv")
print(df3)

{}
  total_bill\ttip\tsex\tsmoker\t\ttime\tsize
0    16.99\t1.01\tFemæle\tNo\tSun\tDinner\t2
1      10.34\t1.66\tMale\tNö\tSun\tDinner\t3
2       21.01\t3.5\tMle\tNo\tSun\tDinner\t3
3    23.68\t3.31\tMale\tNo\tSun\tƳȤѧᖯć𝗱ễ𝑓\t2
4    24.59\t3.61\tFemale\tNo\tSun\tDinner\t4


In [54]:
import pandas as pd
import numpy as np
def check_missing_value(filepath, d = ","):
    missing_values = []
    df = pd.read_csv(filepath, sep = d)
    if df.isnull().values.any():
        (rows,cols) = np.where(pd.isnull(df))
        if rows.any():
            for i in range (len(rows)):
                missing_values.append((rows[i], cols[i]))
    return missing_values

In [55]:
print(check_missing_value("../../tests/resources/test_folder_1/db_errors_test_folder/test_missing_values.csv"))

[(0, 4), (1, 0), (3, 1), (4, 5)]


In [42]:
def check_unknown_numeric_values(filepath, d = ","):
    numeric_columns = ["total_bill", "tip", "size"]
    negative_values = {}
    df = pd.read_csv(filepath, sep = d)
    for col in numeric_columns:
        if df[df[col] < 0 ].values.any():
            negative_values[col] = df[df[col] < 0 ].index.tolist()
    return negative_values

In [43]:
print(check_unknown_numeric_values("../../tests/resources/test_folder_1/db_errors_test_folder/test_unknow_cat_num_values.csv"))

{'total_bill': [3], 'tip': [1]}


In [7]:
#Ignore
#
#
# check if csv file have errors (missing header or bad delimiter)
def check_csv_error(filepath):
    errors = empty_errors
    expected_headers = ["total_bill", "tip", "sex", "smoker", "day", "time", "size"]
    with open(filepath, "r", encoding="utf-8") as file:
        header_line = file.readline().strip()
        data_line = file.readline().strip()
    delimiters = [",", "\t", ";", "|", " "]
    delimiter_counts = {d: header_line.count(d) for d in delimiters}
    detected_delimiter = max(delimiter_counts, key=delimiter_counts.get)
    detected_headers = [h.strip() for h in header_line.split(detected_delimiter)]
    missing_headers = list(set(expected_headers) - set(detected_headers))
    # bad delimiter checker
    if detected_delimiter != ",":
        errors["bad_delimiter"] = detected_delimiter
    # missing column and header checker
    if missing_headers:
        data_columns = data_line.split(detected_delimiter)
        num_columns = max(len(detected_headers), len(data_columns))
        if num_columns < 7:
            errors["missing_columns"] = 7 - num_columns
            # idk if to stop in cases this severe and manually deal with it or just continue to detect errors
        else:
            errors["missing_header"] = missing_headers
    ########################
    # check encoding
    ########################
    ########################
    # start checking for data problems
    # may separate check errors code here, since delimiter is now detected
    df = pd.read_csv(file, delimiter=detected_delimiter)
    # check how many values are missing
    if df.isnull().values.any():
        amount_missing = df.isnull().sum().sum()
        errors["missing_values"] = amount_missing
    ########
    # unknown_categorical_values seem to need comparison with a standard
    # https://www.kaggle.com/code/manishkc06/handling-unknown-categories-in-dataset
    ########
    # bad_data_type_values, unknown_numeric_values: check next block
    return errors


print(
    check_csv_error(
        "../../tests/resources/test_folder_1/db_errors_test_folder/test_bad_csv_errors.csv"
    )
)

{'missing_header': ['day'], 'bad_csv_format': '\t', 'bad_csv_encoding': None, 'missing_columns': None, 'missing_values': None, 'unknown_categorical_values': None, 'unknown_numeric_values': None, 'bad_data_type_values': None}


In [34]:
# check for bad_data_type_values
def check_bad_type_values(filepath, d = ','):
    expected_types = {
        "total_bill": float,
        "tip": float,
        "sex": str,
        "smoker": str,
        "day": str,
        "time": str,
        "size": int,
    }
    df = pd.read_csv(filepath, sep = d)
    bad_type_values = {}
    for col, expected_type in expected_types.items():
        if expected_type == int:
            bad_type_values[col] = df.loc[
                ~df[col].astype(str).str.match(r"^-?\d+$", na=False), col
            ].index.tolist()
        elif expected_type == float:
            bad_type_values[col] = df.loc[
                ~df[col].astype(str).str.match(r"^-?\d+(\.\d+)?$", na=False), col
            ].index.tolist()
        elif expected_type == str:
            bad_type_values[col] = df.loc[
                df[col].astype(str).str.match(r"^-?\d+(\.\d+)?$", na=False), col
            ].index.tolist()

    bad_type_values = {col: vals for col, vals in bad_type_values.items() if vals}
    return bad_type_values

In [36]:
print(check_bad_type_values(
    "../../tests/resources/test_folder_1/db_errors_test_folder/test_bad_data_type.csv"
))
df = pd.read_csv("../../tests/resources/test_folder_1/db_errors_test_folder/test_bad_data_type.csv")
print(df)

{'tip': [2], 'sex': [3], 'smoker': [1, 3, 4], 'day': [0], 'size': [0]}
   total_bill    tip     sex smoker  day    time size
0       16.99   1.01  Female     No    7  Dinner  two
1       10.34   1.66    Male      0  Sun  Dinner    3
2       21.01  three    Male     No  Sun  Dinner    3
3       23.68   3.31       1      1  Sun  Dinner    2
4       24.59   3.61  Female      1  Sun  Dinner    4


In [39]:
# check for unknown_categorical_values
def check_unknown_categorical_values(filepath, d = ","):
    expected_categories = {
        "sex": {"Female", "Male"},
        "smoker": {"Yes", "No"},
        "day": {"Sun", "Sat", "Fri", "Thur", "Wed", "Tue", "Mon"},
        "time": {"Lunch", "Dinner"},
    }

    df = pd.read_csv(filepath, sep = d)
    unknown_cat_values = {}

    for col, expected_set in expected_categories.items():
        if col in df:
            bad_rows = df[~df[col].astype(str).isin(expected_set)]
            if not bad_rows.empty:
                unknown_cat_values[col] = bad_rows.index.tolist()  # Store row indices

    return unknown_cat_values

In [41]:
print(check_unknown_categorical_values("../../tests/resources/test_folder_1/db_errors_test_folder/test_unknow_cat_num_values.csv"))
df1 = pd.read_csv("../../tests/resources/test_folder_1/db_errors_test_folder/test_unknow_cat_num_values.csv")
print(df1)

{'sex': [4], 'smoker': [4], 'time': [1]}
   total_bill   tip         sex smoker  day    time  size
0       16.99  1.01      Female     No  Sun  Dinner     2
1       10.34 -1.66        Male     No  Sun  Drinks     3
2       21.01  3.50        Male     No  Sun  Dinner   300
3      -23.68  3.31        Male     No  Sun   Lunch     2
4       24.59  3.61  Non-binary      1  Sun   Lunch     4


In [37]:
# code to test before putting into checkers
# currently: check bad value type
df = pd.read_csv(
    "../../tests/resources/test_folder_1/db_errors_test_folder/test_bad_data_type.csv"
)
print(df)
df.head()
df.loc[~df["size"].str.isdigit(), "size"].tolist()

   total_bill    tip     sex smoker  day    time size
0       16.99   1.01  Female     No    7  Dinner  two
1       10.34   1.66    Male      0  Sun  Dinner    3
2       21.01  three    Male     No  Sun  Dinner    3
3       23.68   3.31       1      1  Sun  Dinner    2
4       24.59   3.61  Female      1  Sun  Dinner    4


['two']